In [ ]:
# Google scraping settings
FETCH_GOOGLE_LINKS_RETRIES = 1
FETCH_GOOGLE_LINKS_SELECTORS = [
    "div.g a",        # Very generic fallback
]

NO_GOOGLE_SEARCH_RESULTS_MESSAGES = [
    "did not match any documents",
    "no results found for",
    "did not match any web results",
    "no results found for",
    "make sure all words are spelled correctly",
    "try different keywords",
    "try more general keywords",
    "try fewer keywords",
]


In [ ]:
def fetch_google_results(self, query: str) -> tuple[str, bool, list[str]]:
    """Fetch Google search results for an author."""
    url = f"https://www.google.com/search?q={query}"

    links = []
    no_result = False
    retry_count = 0
    while not links and retry_count < FETCH_GOOGLE_LINKS_RETRIES:
        response = send_request(url=url)

        for message in NO_GOOGLE_SEARCH_RESULTS_MESSAGES:
            if message in response.text:
                no_result = True
                break

        soup = BeautifulSoup(response.text, "html.parser")
        for selector in FETCH_GOOGLE_LINKS_SELECTORS:
            results = soup.select(selector)
            if results:
                for result in results:
                    href = result.get("href")
                    if href and href.startswith("http") and "google.com" not in href:
                        links.append(href)
                break  # Stop if we found results
        retry_count += 1

    # Remove any duplicate links and limit to top 10 while keeping the order
    links = list(dict.fromkeys(links))[:10]
    return response.text, no_result, links